In [1]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from sympy import sympify
import sys; sys.path.insert(0, "../"); from utils import *
from models import SympyTorch, PartialDerivativeCalculator, CoeffLearner
from parametric_discovery_pinn import ParametricPINN, BurgerPINN
from madgrad import MADGRAD

from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR
from parametric_pde_diff import TrainSGTRidge, create_groups

from scipy.integrate import odeint
from numpy.fft import fft, ifft, fftfreq
from time import time

from pysr import pysr, best

fontsize = 20

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pickle_load("../parametric_pde_data/parametric_burgers.pkl")

x = data['x']; spatial_dims = x.shape[0]
t = data['t']; time_dims = t.shape[0]

Exact = data['u']
X, T = np.meshgrid(x, t)

X_star = np.hstack((to_column_vector(X), to_column_vector(T)))
u_star = to_column_vector(Exact.T)

# domain bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Sampling training data points
N = 20000
training_idxs = sampling_from_rows(X_star, N, True)
X_train = X_star[training_idxs, :]
u_train = u_star[training_idxs, :]

# to_tensor
X_star = to_tensor(X_star, True)
u_star = to_tensor(u_star, False)
X_train = to_tensor(X_train, True)
u_train = to_tensor(u_train, False)
lb = to_tensor(lb, False)
ub = to_tensor(ub, False)

u_xx_true = 0.1*np.ones(time_dims)
uu_x_true = -1*(1+0.25*np.sin(t))

feature_names = ['u', 'u_x', 'u_xx']

Loaded from ../parametric_pde_data/parametric_burgers.pkl


In [3]:
pinn = ParametricPINN(scale=False, lb=lb, ub=ub)
print("Loaded the pretrained weights")
pinn.load_state_dict(torch.load("./saved_path_inverse_parametric_burgers/parametric_pinn.pth"))

Loaded the pretrained weights


<All keys matched successfully>

In [4]:
expression, _ = build_exp("-0.1872898*sin(t)-1.0238724")
mod = SympyTorch(expressions=[expression])

Please ensure that all coefs are different


In [5]:
t = torch.rand(100, 1)
mod({'t':t})

tensor([[-1.1594],
        [-1.1564],
        [-1.1655],
        [-1.1593],
        [-1.1475],
        [-1.1713],
        [-1.1814],
        [-1.1792],
        [-1.1430],
        [-1.1568],
        [-1.0794],
        [-1.1031],
        [-1.1542],
        [-1.0833],
        [-1.1187],
        [-1.0665],
        [-1.0771],
        [-1.1056],
        [-1.1420],
        [-1.1128],
        [-1.1277],
        [-1.0452],
        [-1.0659],
        [-1.0568],
        [-1.1247],
        [-1.1181],
        [-1.0646],
        [-1.1157],
        [-1.0509],
        [-1.1309],
        [-1.0963],
        [-1.1274],
        [-1.0327],
        [-1.1571],
        [-1.1043],
        [-1.1149],
        [-1.0696],
        [-1.1798],
        [-1.1707],
        [-1.1394],
        [-1.1343],
        [-1.1010],
        [-1.1578],
        [-1.1176],
        [-1.1290],
        [-1.1213],
        [-1.1713],
        [-1.1120],
        [-1.0260],
        [-1.1804],
        [-1.1350],
        [-1.0253],
        [-1.

In [6]:
model = nn.Sequential(pinn.preprocessor_net, pinn.pde_solver_net)

In [7]:
xx, tt = dimension_slicing(X_train)
u = model(cat(xx, tt))
gradients_dict(u, xx, tt, feature_names)

{'u': tensor([[0.0322],
         [0.1881],
         [0.0002],
         ...,
         [0.0005],
         [0.0575],
         [0.0960]], grad_fn=<AddmmBackward>),
 'u_x': tensor([[ 1.0749e-01],
         [ 7.1286e-02],
         [-7.6606e-05],
         ...,
         [-1.6100e-04],
         [-2.1388e-01],
         [ 1.2128e-01]], grad_fn=<SliceBackward>),
 'u_xx': tensor([[ 1.0749e-01],
         [ 7.1286e-02],
         [-7.6606e-05],
         ...,
         [-1.6100e-04],
         [-2.1388e-01],
         [ 1.2128e-01]], grad_fn=<SliceBackward>)}

In [8]:
mod1, variables1 = string2sympytorch("u*u_x")
mod1(gradients_dict(u, xx, tt, sorted(list(map(str, variables1)))))

Please ensure that all coefs are different


tensor([[ 3.4579e-03],
        [ 1.3410e-02],
        [-1.6341e-08],
        ...,
        [-7.7689e-08],
        [-1.2307e-02],
        [ 1.1640e-02]], grad_fn=<SqueezeBackward1>)

In [9]:
mod2, variables2 = string2sympytorch("u_xx")
mod2(gradients_dict(u, xx, tt, sorted(list(map(str, variables2)))))

Please ensure that all coefs are different


tensor([[ 1.0749e-01],
        [ 7.1286e-02],
        [-7.6606e-05],
        ...,
        [-1.6100e-04],
        [-2.1388e-01],
        [ 1.2128e-01]], grad_fn=<SqueezeBackward1>)

In [10]:
pdc = PartialDerivativeCalculator(["u*u_x", "u_xx"], ["-0.1872898*sin(t)-1.0238724", "0.09875935"])

Please ensure that all coefs are different
Please ensure that all coefs are different
Please ensure that all coefs are different
Please ensure that all coefs are different


In [11]:
pdc(u, xx, tt)

tensor([[-3.5658e-03,  1.0616e-02],
        [-1.3566e-02,  7.0401e-03],
        [ 1.9781e-08, -7.5656e-06],
        ...,
        [ 8.1879e-08, -1.5900e-05],
        [ 1.4352e-02, -2.1123e-02],
        [-1.3273e-02,  1.1978e-02]], grad_fn=<CatBackward>)